# Azure AI Agent Service 및 Semantic Kernel을 사용한 RAG

이 코드 스니펫은 `Azure AI Agent Service`와 `Semantic Kernel`을 사용하여 검색 기반 생성(RAG)을 위한 Azure AI 에이전트를 생성하고 관리하는 방법을 보여줍니다. 에이전트는 검색된 컨텍스트를 기반으로 사용자 질문을 처리하고 이에 따라 정확한 응답을 제공합니다.


SQLite 버전 수정  
다음 오류가 발생할 경우:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

노트북의 시작 부분에서 이 코드 블록의 주석을 해제하세요:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### 패키지 가져오기
다음 코드는 필요한 패키지를 가져옵니다:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Semantic Kernel 및 Azure AI Agent Service를 활용한 Retrieval-Augmented Generation

이 샘플은 **Azure AI Agent Service**를 사용하여 업로드된 문서의 도메인별 컨텍스트와 언어 모델을 결합하여 **Retrieval-Augmented Generation (RAG)**을 수행하는 방법을 보여줍니다.

### 작동 방식

1. **문서 업로드**: 정보(Contoso의 여행 보험 정책)가 포함된 markdown 파일(document.md)이 에이전트 서비스에 업로드됩니다.

2. **벡터 스토어 생성**: 문서가 벡터 스토어에 색인화되어 문서 내용을 기반으로 한 시맨틱 검색이 가능해집니다.

3. **에이전트 구성**: `gpt-4o` 모델을 사용하여 다음과 같은 엄격한 지침으로 에이전트를 생성합니다:
   - 문서에서 검색된 내용에 기반하여 질문에 답변합니다.
   - 질문이 범위를 벗어나는 경우 답변을 거부합니다.

4. **파일 검색 도구 통합**: 에이전트에 `FileSearchTool`이 등록되어, 모델이 추론 중에 색인화된 문서에서 관련된 스니펫을 검색하고 가져올 수 있습니다.

5. **사용자 상호작용**: 사용자는 질문을 할 수 있습니다. 문서에서 관련 정보를 찾으면 에이전트가 근거 있는 답변을 생성합니다.  
   관련 정보가 없으면 에이전트는 문서에 충분한 정보가 없다고 명확히 응답합니다.


### 주요 함수

```python
def main():
    # 프로그램의 진입점
    print("Hello, World!")
```

### 설명

이 함수는 프로그램의 시작점으로, "Hello, World!"를 출력합니다.

### 사용법

1. 코드를 실행합니다.
2. 출력 창에서 "Hello, World!" 메시지를 확인합니다.

### 주의사항

- 이 함수는 기본적인 예제이며, 실제 애플리케이션에서는 더 복잡한 로직이 필요할 수 있습니다.
- 프로그램을 실행하기 전에 Python이 설치되어 있는지 확인하세요.

### 추가 정보

이 예제는 Python 프로그래밍 언어를 사용하여 작성되었습니다. Python에 대한 자세한 정보는 [Python 공식 문서](https://www.python.org/doc/)를 참조하세요.


Azure CLI를 사용하여 `az login`을 먼저 실행하여 `DefaultAzureCredential`을 사용할 때 적절한 인증 컨텍스트가 제공되도록 하십시오. Azure AI Agent Service는 API 키를 사용하지 않습니다.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

물론입니다! 번역된 내용을 아래에 제공합니다:

---

# 소개

이 문서는 @@INLINE_CODE_0@@를 사용하는 방법에 대한 기본적인 개요를 제공합니다. 이 도구는 개발자들이 더 효율적으로 작업할 수 있도록 설계되었습니다.

[!NOTE] 이 문서는 초보자와 숙련된 사용자 모두를 대상으로 합니다.

---

## 설치

@@INLINE_CODE_1@@를 설치하려면 다음 단계를 따르세요:

1. @@INLINE_CODE_2@@를 다운로드합니다.
2. @@INLINE_CODE_3@@를 실행하여 설치를 시작합니다.
3. 설치가 완료되면 @@INLINE_CODE_4@@를 확인하세요.

[!TIP] 설치 중 문제가 발생하면 공식 문서를 참조하세요: @@INLINE_CODE_5@@

---

## 사용법

### 기본 사용법

@@INLINE_CODE_6@@를 실행하려면 다음 명령을 사용하세요:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

이 명령은 기본 설정으로 프로그램을 실행합니다.

[!WARNING] 명령을 실행하기 전에 모든 필수 구성 요소가 설치되었는지 확인하세요.

---

### 고급 사용법

고급 옵션을 사용하려면 다음을 참고하세요:

@@CODE_BLOCK_1@@

이 명령은 사용자 정의 설정을 적용합니다.

[!CAUTION] 잘못된 설정은 프로그램이 예상대로 작동하지 않을 수 있습니다.

---

## 문제 해결

문제가 발생하면 다음 단계를 따르세요:

1. 로그 파일을 확인합니다: @@INLINE_CODE_7@@
2. 오류 메시지를 분석합니다.
3. 공식 지원 포럼에서 도움을 요청합니다: @@INLINE_CODE_8@@

[!IMPORTANT] 문제를 보고할 때는 가능한 한 많은 세부 정보를 제공하세요.

---

## 결론

이 문서를 통해 @@INLINE_CODE_0@@의 기본적인 사용법을 익히셨기를 바랍니다. 추가 정보는 공식 문서를 참조하세요: @@INLINE_CODE_9@@

[!TIP] 새로운 업데이트가 있을 수 있으니 정기적으로 확인하세요.

---



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
